In [1]:
!pip3 install "transformers[torch]"
!pip3 install datasets
!pip3 install seqeval
!pip3 install scikit-learn
!pip3 install evaluate


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from datasets import load_from_disk
import torch
import lib
import numpy as np
import evaluate
import seqeval
from transformers import GenerationConfig

In [3]:
train_dataset = load_from_disk('full_valid_dataset_sentence-81583/train')
test_dataset = load_from_disk('full_valid_dataset_sentence-81583/test')
eval_dataset = load_from_disk('full_valid_dataset_sentence-81583/eval')
model_name = 'bert-large-uncased'

In [4]:
num_labels = len(lib.id2label)
max_length = 64

gen_config = GenerationConfig(max_length=max_length)

tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

new_tokens = ['[MATH]', '[DISPLAY]', '[ITALIC]']
num_added_tokens = tokenizer.add_tokens(new_tokens)

model.resize_token_embeddings(len(tokenizer))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30525, 1024)

In [5]:
def retokenize(row):
    tokens = ['[CLS]'] + row['tokens'][:max_length - 2] + ['[SEP]']
    annotations = ['O'] + row['spacy'][:max_length - 2] + ['O']
    remaining = max_length - (len(tokens))
    input_ids = tokenizer.convert_tokens_to_ids(tokens + ['[PAD]'] * remaining)
    
    row['input_ids'] = input_ids
    row['attention_mask'] = [1 if token_id != tokenizer.pad_token_id else 0 for token_id in input_ids]
    row['labels'] = [lib.label2id[label] for label in annotations] + [-100] * remaining
    return row
    
train_dataset = train_dataset.map(retokenize)
test_dataset = test_dataset.map(retokenize)
eval_dataset = eval_dataset.map(retokenize)

In [6]:
train_dataset

Dataset({
    features: ['file_name', 'title', 'tokens', 'math_index', 'nltk_annotate', 'spacy', 'is_valid', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 65266
})

In [7]:
metric = evaluate.load('seqeval')
def compute_metrics(p, all_metrics=False):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [lib.id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [lib.id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if all_metrics:
        return results

    noun = results["OUN"]["f1"]
    adj = results["DJ"]["f1"]

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
        "Noun f1": results["OUN"]["f1"],
        "ADJ f1": results["DJ"]["f1"],
        "Harmonic mean": (2 * noun * adj) / (noun + adj)
    }



In [8]:
if torch.backends.mps.is_available():
    print("Using mps")
    mps_device = torch.device("mps")
    model.to(mps_device)

if torch.cuda.is_available():
    print("Using cuda")
    cuda_device = torch.device("cuda")
    model.to(cuda_device)

Using cuda


In [9]:


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=2000,
    weight_decay=0.013,
    label_smoothing_factor=0.009,
    learning_rate=0.0001,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='Harmonic mean',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
print("Starting training")
trainer.train()
print("Ending training")

Starting training


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Noun f1,Adj f1,Harmonic mean
1,0.193500,0.189461,0.948624,0.950986,0.949803,0.960865,0.915634,0.886399,0.900780
2,0.183100,0.176462,0.955025,0.955563,0.955294,0.965460,0.924982,0.891550,0.907958
3,0.160100,0.161520,0.961594,0.960753,0.961173,0.970151,0.934155,0.908087,0.920936
4,0.138200,0.155515,0.965887,0.965897,0.965892,0.973845,0.942194,0.916368,0.929101
5,0.119200,0.148902,0.970658,0.969799,0.970228,0.977208,0.948699,0.927872,0.938170
6,0.118500,0.152148,0.971657,0.970489,0.971073,0.977793,0.949601,0.931873,0.940653
7,0.099800,0.153572,0.973894,0.973655,0.973774,0.979862,0.954629,0.937176,0.945822
8,0.096400,0.153185,0.974576,0.974815,0.974696,0.980562,0.956370,0.937366,0.946773
9,0.089900,0.156247,0.975128,0.975368,0.975248,0.981055,0.957055,0.939831,0.948365
10,0.083500,0.155084,0.976968,0.976288,0.976628,0.982062,0.958942,0.942126,0.950459


Ending training


In [12]:
model.save_pretrained('./trained_bert_model_full_data_large')

In [13]:
predictions = trainer.predict(eval_dataset)

model_predictions = predictions.predictions

true_label_ids = predictions.label_ids

p = (model_predictions, true_label_ids)
all_metrics = compute_metrics(p, True)

In [14]:
all_metrics

{'ART': {'precision': 0.9914015477214101,
  'recall': 0.9812765957446808,
  'f1': 0.9863130881094953,
  'number': 1175},
 'CONJ': {'precision': 0.991674595623216,
  'recall': 0.9959388437649307,
  'f1': 0.9938021454112037,
  'number': 4186},
 'DJ': {'precision': 0.9389877513288652,
  'recall': 0.9491881789510571,
  'f1': 0.9440604124310195,
  'number': 8561},
 'DP': {'precision': 0.9952171791117619,
  'recall': 0.9970663015841972,
  'f1': 0.9961408822236334,
  'number': 10226},
 'DV': {'precision': 0.9843798812870977,
  'recall': 0.9788754271512892,
  'f1': 0.981619937694704,
  'number': 3219},
 'ERB': {'precision': 0.982302945628129,
  'recall': 0.9799070847851336,
  'f1': 0.981103552532124,
  'number': 8610},
 'ET': {'precision': 0.9989762489762489,
  'recall': 0.9992831541218637,
  'f1': 0.9991296779808528,
  'number': 9765},
 'NTJ': {'precision': 0.921875,
  'recall': 0.8676470588235294,
  'f1': 0.893939393939394,
  'number': 68},
 'OUN': {'precision': 0.9594136246047715,
  'recall